In [ ]:
from scipy import stats,special
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from random import choices
from glob import glob
import xarray as xr
import datetime as dt
from scipy.stats import multivariate_normal
import warnings
warnings.filterwarnings("ignore")

# Probability Exercises Notebook
#### **Author:** Sean Crowell
#### **Date:** 06/14/2025
#### **Purpose:** a limited set of exercises to refresh basic understanding of random variables, distributions, linear regression, and Bayes' Theorem. This should whet your appetite for the Summer School, where the concepts here will be treated as standard vocabulary. This notebook is roughly patterned after the excellent presentations by Dr. Michael Bertolacci, found here: XXX
#### **License:** Free and open, but please acknowledge and keep this block when sharing. 
#### **Contact:** Please submit feedback to sean@belumenus.com. 
#### **External File:** airquality2.csv, which accompanies the R software package. More details here: https://www.rdocumentation.org/packages/datasets/versions/3.6.2/topics/airquality

In [ ]:
airquality_file='./airquality2.csv'

## **Univariate Probability Distributions**

#### A probability distribution $P$ is a model for our knowledge about one or more given variable(s) $X$ in the context that there are "random" variations in $X$ that we can't explain, and so we call $X$ a **random variable** as the value of $X$ may depend on unseen "random" factors. **Note that "random" doesn't mean "without structure"**. Most of the effort of an undergraduate probability and statistics course is directed towards characterizing different families of distributions to model the randomness we see in real datasets.

#### One way of describing a distribution is its *Probability Density Function* (PDF) $p(x)$. The integral of the PDF, $\int_a^b p(x) dx$, gives the probability that $X$ falls in a certain interval $[a,b]$. 
#### The *mean* is given by $$\mu := E[x]:=\int_A xp(x) dx$$ and *variance* by  $$\sigma^2 := E[(x-\mu)^2] := \int_A (x-\mu)^2 p(x) dx.$$ These numbers tell us about where the distribution is "centered" and how "spread out" it is. Sometimes we refer to square root of the variance, $\sigma$, called the *standard deviation*.
#### Technical note: the set $A$ in these integrals consists of all the values that make sense to evaluate $p(x)$, called the *support* of $p$.

### **Gaussian Distribution**
#### The **Gaussian** (or Normal) distribution is the most important probability distribution. It is defined completely by its mean $\mu$ and the variance $\sigma^2$ and has PDF: $$p(x;\mu,\sigma^2) = \frac{1}{\sigma\sqrt{2\pi}}e^\frac{-(x-\mu)^2}{2\sigma^2}.$$

---
This is just a convenience function so we don't have to rewrite the formula over and over.

`mu` is the mean

`sig` is the standard deviation

In [ ]:
def gaus_pdf(x,mu=0,sig=1):
    return np.exp(-(x-mu)**2/sig**2/2)/sig/np.sqrt(2*np.pi)

#### You can define a one variable function by specifying a single mean and standard deviation with the code 
#### `gpdf = lambda x: gaus_pdf(x,mu=mean_value,sig=stdev_value)`
---
### Example: Gaussian PDFs with different means and standard deviations

In [ ]:
# Specify the mean and standard deviation:
mu_0 = -2.0
sig_0 = 0.5
gpdf_0 = lambda x: gaus_pdf(x,mu=mu_0,sig=sig_0)
# 
mu_1 = 1
sig_1 = 2
gpdf_1 = lambda x: gaus_pdf(x,mu=mu_1,sig=sig_1)

x = np.linspace(-20,20,401)
fig,ax = plt.subplots(1,1)
g0 = ax.plot(x,gpdf_0(x))
g1 = ax.plot(x,gpdf_1(x))
ax.set_title('Univariate Gaussian PDFs')
ax.legend([g0[0],g1[0]],[r'$\mu = -2$, $\sigma^2 = 1$',r'$\mu = 0.5$, $\sigma^2 = 4$']);

#### Questions
1. What happens to the PDF as we make `mu` larger or smaller?
2. What happens to the PDF as we make `sig` larger or smaller?

#### We specified the mean `mu` and standard deviation `sig` using the convenience function, but does it match the definition above using the integral? The numpy function `trapz` is a handy simple way to compute an integral numerically.

In [ ]:
#Compute the mean and standard deviation with trapz for the first Gaussian PDF
mu_s = np.trapz(x*gpdf_0(x),x)
sig_s = np.sqrt(np.trapz((x-mu_0)**2*gpdf_0(x),x))

print(f"Specified Mean = {mu_0}, standard deviation = {sig_0}")
print(f"Computed mean = {mu_s}, standard deviation = {sig_s}")

#### Not bad! What about the second Gaussian that we called `gpdf_1`? 

In [ ]:
#Copy and paste the code above and change it to compute the mean and standard deviation for gpdf_1


---

### Sample Distributions vs. PDFs
#### Observational data is usually a collection of samples. We can use simple tools like histograms to model the distribution of the samples with a theoretical distribution like a Gaussian.
#### Simple example: The air quality dataset referenced in the notes gives us a good opportunity to explore different distributions. The reference is here: https://www.rdocumentation.org/packages/datasets/versions/3.6.2/topics/airquality

In [ ]:
def read_air_quality_data(file=None):
    if file is not None:
        df = pd.read_csv(file)
        dates = np.array([dt.datetime(1973,df['Month'].iloc[i],df['Day'].iloc[i]) for i in range(len(df['Month']))])
        df.index = dates
        df['logOzone'] = np.log(df['Ozone'])
        del df['Ozone']
        del df['Month']
        del df['Day']
    return df

In [ ]:
df = read_air_quality_data(file=airquality_file)
mu = df.mean()
sig = df.std()
fig,axs = plt.subplots(2,2,figsize=(12,12))
for iky,ky in enumerate(sorted(df.keys())):
    ax = axs[iky//2,iky%2]
    gpdf = lambda x: gaus_pdf(x,mu[ky],sig[ky])
    df[ky].hist(ax=ax)
    xl = ax.get_xlim()
    yl = ax.get_ylim()
    x = np.linspace(xl[0],xl[1],21)
    ax1 = ax.twinx()
    ax1.plot(x,gpdf(x),color='tab:orange')
    ax.set_title(ky)
    

#### Questions
1. Qualitatively, how well does the Gaussian density curves match the histograms?
2. Qualitatively, how well do the mean and standard deviation describe the distribution of Solar Radiation, Temperature, Wind Speed, and logOzone during this time period?

## **Ordinary Least Squares (OLS) Linear Regression**
#### Observational scientists often hypothesize about the relationships between different measured quantities using statistical models. The simplest such model is a 1-D linear function: $$y = \alpha x + \beta + \epsilon.$$ Here $y$ is the observed variable we want to predict, $\alpha$ and $\beta$ are the slope and intercept parameters of the linear function, and $\epsilon$ is a random variable that captures the assumed noise/error distribution in the observations and model. In the simplest case, we assume $\epsilon\sim N(0,\sigma^2)$ for some variance $\sigma^2$.
#### If we have a collection of observations $y_1,y_2,...$ that go with inputs $x_1,x_2,...$, then we would like to choose $\alpha$ and $\beta$ so that the squared error between the predictions $\hat{y}_i = \alpha x_i + \beta$ and the observations is minimized. $\epsilon$ is ignored during this fitting process, but is a **critical** part of the assumptions for the OLS linear model to give trustworthy predictions. 
---
### **Example: Ozone and Solar Radiation**
#### It is well-understood that solar radiation and ozone production are related to one another. We want to see if there is a clear relationship in these data. 
#### Let's do some exploratory data analysis to see if a linear model would make sense. The first step is a simple scatter plot.

In [ ]:
df = read_air_quality_data(file=airquality_file)
fig,ax = plt.subplots(1)
g1 = ax.scatter(df['Solar.R'],df['logOzone'])
ax.set_title('LogOzone versus Solar Radiation')
ax.set_ylabel('log(Ozone in ppb) ');
ax.set_xlabel('Solar Radiation (langleys)');

#### The scatterplot "looks like" a line could pass nicely through the points and capture the upward trend, which suggests that we could use a linear function to predict FC temperatures from the ongoing temperature record in Denver.
#### To find the optimal parameters $\alpha$ and $\beta$ in our linear model, we will use the Python library `statsmodels` (but we could  easily solve the basic matrix equations ourselves):

In [ ]:
import statsmodels.api as sm
X = df['Solar.R']        # Predictors
y = df['logOzone']       # Observations
X = sm.add_constant(X)   # Allows for an offset
ols = sm.OLS(y, X).fit() # Fit the model
ols.summary()            # Print the results of the OLS

#### Whew! That's a lot of information. Let's summarize the important bits:
1. The OLS model is logOzone = 2.7392 + 0.0038 Solar.R.
2. The $R^2$ value, which is one measure of goodness-of-fit, is 0.180. This roughly means that this model explains 18% of the variability in the ozone observations (so something else must be causing much of the variability).
3. The t-statistic tells us that the coefficients in our model can be trusted to be different from zero, i.e., that some degree of linear relationship exists.

#### Let's plot the fitted line through the data and the histogram of the residuals.

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(10,4))

df['logOzone_ols'] = ols.fittedvalues
ax = axs[0]
df.plot.scatter(x='Solar.R',y='logOzone',ax=ax)
df.plot(x='Solar.R',y='logOzone_ols',ax=ax,color='tab:orange')

ax = axs[1]
ols.resid.hist(ax=ax)
#ax.legend(['OLS with Intercept','OLS without Intercept'])
ax.set_xlabel('Observed - Predicted logOzone')

#### The predictions are on the left panel and the residuals (Observed - Predicted logOzone). 
---
#### Questions:
1. Does a linear model seem appropriate for this dataset?
2. Do the residuals seem to be Gaussian distributions? What are the means and variances?

## **Multivariate Probability Distributions**

#### As our observations grow to multiple variables, we start to consider how our knowledge of those variables might be connected, where "knowledge" is a vague term that we typically use as shorthand for  probability distributions. 
---
#### We saw in the last example that Solar Radiation and Ozone concentration seem to have a weak linear relationship. We can examine the other variables as well. 

In [ ]:
df = read_air_quality_data(file=airquality_file)
sns.pairplot(df)

#### This matrix of plots shows the distributions of each variable as histograms along the diagonal and scatter plot of each pair of variables. The bottom row shows the ozone concentration as a function of the Solar Radiation, Wind Speed, and Temperature. 

#### Questions
1. What does the pattern of dots tell us about the relationship between ozone and the other three variables?
2. How about the other variables, e.g. temperature and wind speed?
---

### **Joint Probability Distributions**
#### We describe full state of knowledge of multiple variables as the **joint probability distribution**. We write (for two variables) as $Z = (X,Y)$ with distribution $P_Z$. As with the 1-D discussion above, we can discuss the **joint probability density function** $p(x,y)$ that we use in a similar fashion to compute probabilities, only now with double integrals. We also define the **marginal density function** for $x$: $$ p(x) = \int p(x,y) dy $$ and similarly for $p(y)$.

#### Generalizing other concepts from the 1-D case:
#### The mean $\mu = [\mu_X,\mu_Y]$ is just the variable-wise mean of the marginal density functions. 
#### The covariance between real-valued RVs $X$ and $Y$ is defined by
#### $$cov(X,Y) = E[(X-\mu_X)(Y-\mu_Y)]$$ (now a double integral over x and y).
#### Note that $cov(X,X) = \sigma_X^2$ and $cov(X,Y) = cov(Y,X)$. The correlation between $X$ and $Y$ is covariance scaled by the individual standard deviations:
#### $$\rho(X,Y) = \frac{cov(X,Y)}{\sigma_X\sigma_Y}.$$ Note that $\rho(X,X) = 1$, $\rho(X,Y) = \rho(Y,X)$ and $-1 \leq \rho(X,Y) \leq 1.$ We will often refer to the covariance matrix:
#### $$ \Sigma_{XY} = \begin{pmatrix} \sigma_X^2 & cov(X,Y) \\ cov(X,Y) & \sigma_Y^2  \end{pmatrix} $$ and the correlation matrix: $$ R_{XY} = \begin{pmatrix} 1 & \rho(X,Y) \\ \rho(X,Y) & 1  \end{pmatrix} $$

---
### **Air Quality Data Example:** 
#### Let's compute the covariance matrix and correlation for the variables in the air quality dataset.

In [ ]:
df = read_air_quality_data(file=airquality_file)

mu = df.mean()
cv = df.cov() #Pandas computes this easily
cr = df.corr()
fig,ax = plt.subplots(1)
sns.heatmap(cr,ax=ax,cmap=plt.cm.RdBu_r,vmin=-1,vmax=1)
xl = ax.get_xlim()
yl = ax.get_ylim()
for x in [1,2,3]:
    ax.plot(np.array([x,x]),yl,'--k')
    ax.plot(xl,np.array([x,x]),'--k')
ax.set_title('Correlation Matrix for \n Air Quality Dataset');


#### The blue colors indicate a negative correlation (anticorrelation), while red colors indicate a positive correlation. 

#### We note a few things: 
- Solar radiation and Temperature are correlated with ozone, while wind speed is anticorrelated with all the other variables. 
- The strongest correlation is between temperature and logOzone.
- The strongest anticorrelation is between wind and logOzone.
- Wind and temperature are also anticorrelated.

#### Questions:
1. Compare the correlation matrix with the scatter plots above. What does the negative correlation correspond to in the datasets?
2. How about the positive correlation?
3. What does the strength of the correlation correspond to?
---

---
### Multivariate Gaussian Distribution
#### With these definitions in hand, we are ready to write down the joint density for the **multi-variate Gaussian** distribution with random vector $Z=(X,Y)$, mean vector $\mu=(\mu_X,\mu_Y)$, and covariance matrix $\Sigma$: $$ f(z; \mu,\Sigma)= (2\pi)^{-1}|\Sigma|^{-1/2}\exp\left(-\frac{1}{2}(z - \mu)^{T}\Sigma^{-1}(z-\mu)\right)$$ where $|\Sigma|$ is the determinant of the covariance matrix.
#### **Important Notes**
- The quantity $\Sigma^{-1}(z-\mu)$ is actually a matrix-vector product since $z$ and $\mu$ are 2-D vectors. In fact, it will be a new column vector with two entries.
- The entire quantity $(z - \mu)^{T}\Sigma^{-1}(z-\mu)$ is a scalar, since $(z - \mu)^{T}$ is a 1 x 2 row vector.
- So even though we are talking about multivariate distributions, they are still predicting a number, i.e., the value of $Z$ is a real number.

### 2-D Examples
#### Let's use our favorite temperature dataset and examine the distributions for each pair of locations. We will plot 2D histograms as well as 2D Gaussian Densities with the same mean and covariance. 

In [ ]:
df = read_air_quality_data(file=airquality_file)

#compute means and covariances
mu = df.mean()
cv = df.cov()

fig,axs = plt.subplots(2,3,figsize=(16,8))
for iky,ky in enumerate(['Solar.R','Temp','Wind']):
    mu_s = np.array([mu[ky],mu['logOzone']])
    cv_s = np.cov(df[ky],df['logOzone'])
    x,y = np.meshgrid(np.linspace(df[ky].min(),df[ky].max(),21),np.linspace(df['logOzone'].min(),df['logOzone'].max(),21))
    pos = np.dstack((x,y))
    # Define the PDFs
    pdf = multivariate_normal(mu_s,cv_s).pdf(pos)

    ax = axs[0,iky]
    g = ax.hist2d(df[ky],df['logOzone'],cmap=plt.cm.hot_r)
    ax.set_title(f'2D Histogram of {ky} and logOzone ',fontsize=16);
    ax.set_xlabel(ky);
    ax.set_ylabel('logOzone');
    plt.colorbar(g[-1],ax=ax)
    
    ax = axs[1,iky]
    g = ax.contourf(x,y,pdf,cmap=plt.cm.hot_r)
    ax.set_title(f'Gaussian Joint Density of \n {ky} and logOzone',fontsize=16);
    ax.set_xlabel(ky);
    ax.set_ylabel('logOzone');
    plt.colorbar(g,ax=ax)

plt.tight_layout()

#### Questions
1. How well do the mean and covariance describe each of the sample joint distributions between logOzone and the other variables? If they do a good job, the sample histogram should look a lot like its Gaussian density. 
2. Would you use a Gaussian distribution as a model for the joint distribution of logOzone with any of the other variables? Which ones?
----

## **Multiple Linear Regression**
#### We can generalize the OLS concepts to multiple predictors: $$y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_n x_n + \epsilon.$$ Just like before, the $\epsilon \sim N(0,\sigma^2)$ is a random variable that represents the error in modeling $y$ arising "noise" in the data. Observations are tuples $(\hat{x}_1,...,\hat{x}_n)$ with a coincident observed value $\hat{y}$. We want to find a vector $\beta = (\beta_1,...\beta_n)$ so that the predicted $y$ values are as close to the observations as possible. Practically this is very similar to the OLS problem  above, just with more unknowns. We solve a similar linear system of equations. 
---
### Example: Predicting ozone with solar radiation, temperature, and wind
#### Looking at the scatter plots above, we guess that a linear model could explain ozone variability in terms of the variability in the other variables. Let's use `statsmodels` again to derive a linear model. The high correlation between temperature and wind speed should give us pause - it can lead to multicollinearity as discussed in the lectures. 

In [ ]:
import statsmodels.api as sm
df = read_air_quality_data(file=airquality_file)
X = np.column_stack((df['Solar.R'],df['Temp'],df['Wind']))        # Predictors
y = df['logOzone']       # Observations
ols = sm.OLS(y, X).fit() # Fit the model
ols.summary()            # Print the results of the OLS

Let's summarize the important bits:
1. The OLS model is $$ \mbox{logOzone} = 0.0022 \mbox{Solar.R} + 0.0471 \mbox{Temp} - 0.0642 \mbox{Wind} $$
2. The $R^2$ value, which is one measure of goodness-of-fit, is 0.983. That's significantly better than the OLS model with Solar.R alone.
3. The t-statistics on the coefficients tell us that the coefficients in our model can be trusted to be different from zero, i.e., that some degree of linear relationship exists.

## Conditional Distributions
#### In the setting of individual probability events $A$ and $B$, we define conditional probability as $$P(A|B) = \frac{P(A\cap B)}{P(B)}.$$ We interpret this as "the probability A will occur assuming that B has occurred". 

#### The conditional probability density $p(y|x)$ for two RVs $X$ and $Y$ can be defined similarly with the joint density $p(x,y)$ and the marginal density $p(x)$: $$ p(y|x) = \frac{p(x,y)}{p(x)}.$$ Conceptually we think about this probability in similar terms, but of course computing probabilities requires integrating over the appropriate intervals of interest. 

#### This definition of $p(y|x)$ is not always straightforward to compute in practice, because it requires us to know the joint distribution. Luckily, we can derive a more straightforward tool for this purpose: Bayes' Theorem.

### **Bayes' Theorem**
#### If we repeat the conditional density formula for $p(x|y)$ and solve for $p(x,y)$, we get Bayes' Theorem: $$ p(x|y) = \frac{p(y|x)p(x)}{p(y)} \propto p(y|x)p(x)$$ We think about this equation as direction for using observations $y$ to enhance our understanding of a parameter $x$ from a previous state of knowledge that we call the **prior distribution** with density $p(x)$.  The distribution $p(y|x)$ is usually called the **likelihood**. The distribution $p(x|y)$ is called the **posterior distribution** in the context of updating our knowledge about the parameter $x$ using the prior and likelihood information.
---

## Bayesian Linear Models
#### Following the lecture, we want to write $$ Y = X\beta + \epsilon, $$ where the X matrix is populated by the predictor variables, $\beta$ is the vector of coefficients, and $\epsilon \sim N(0,\Sigma_\epsilon)$ is the observation uncertainty with mean 0 and covariance $\Sigma_\epsilon$.
#### The Bayesian approach to this is to recast $\beta$ as a random vector with some prior knowledge that is updated using observations $\hat{y}$. If we assume that $\beta \sim N(\mu_\beta,\Sigma_\beta)$, we can write the exact form of the posterior distribution $$p(\beta|Y) = \frac{p(Y|\beta)p(\beta)}{p(Y)}.$$ The posterior distribution in this case is actually Gaussian as well, with covariance $$ \widehat{\Sigma}_\beta =\left( (X^\mathsf{T}\Sigma_{\epsilon}^{-1}X)^{-1} + \Sigma_{\beta}^{-1}\right)^{-1} $$ and mean $$ \hat{\beta} = \widehat{\Sigma}_\beta \left(X^\mathsf{T}\Sigma_{\epsilon}^{-1}Y + \Sigma_{\beta}^{-1}\mu_\beta \right). $$ This is an exact solution, so we can compute these directly if we have enough computing power and clever methods.


---
### Example: Air Quality Data
#### Following the lecture, we seek a model form $$ \mbox{logOzone} = \beta_0 + \beta_1 \mbox{Solar.R} + \beta_2 \mbox{Wind} - \beta_3 \mbox{Temp}. $$ Note that we did this above with the OLS approach, meaning that we wanted **the $\beta$** that minimize the squared errors in predictions versus our observations. This time we're going to follow a Bayesian approach. Hence we need to specify  
1. the prior distribution with mean $\mu_\beta$ and covariance $\Sigma_\beta$
2. the observational uncertainty distribution
#### 1. We choose $\mu_\beta = (0,0,0,0)$ and $\Sigma_\beta = 10^2\mathbf{I}.$This large variance (relative to the size of the OLS slopes we saw in the example above) means that we call these "uninformative priors." The mean being 0 is somewhat of a "null hypothesis" that there is no deviation from zero - i.e., there is not a relationship. Also there is no correlation in the prior between the parameters.
#### 2. We set $\Sigma_\epsilon = 0.467^2 \mathbf{I}$ as is done in the lecture.
#### Then just compute the matrix products using `np.dot`.

In [ ]:
df = read_air_quality_data(file=airquality_file)
# Define the prior distribution
prior_mean = np.zeros(4)
prior_cov = 10**2*np.eye(4)
# Observation error covariance
obs_cov = 0.467**2*np.eye(len(df['logOzone']))

#Precompute the matrix inverses
prior_cov_inv = np.linalg.inv(prior_cov)
obs_cov_inv = np.linalg.inv(obs_cov)

In [ ]:
# Compute the matrix solution for the posterior covariance
X = np.column_stack((np.ones(len(df['logOzone'])),df['Solar.R'],df['Wind'],df['Temp']))
Xt_Sige_X = np.dot(X.T,np.dot(obs_cov_inv,X))
post_cov = np.linalg.inv(Xt_Sige_X+prior_cov_inv)
post_corr = np.zeros(post_cov.shape)
for i in range(post_corr.shape[0]):
    for j in range(post_corr.shape[1]):
        post_corr[i,j] = post_cov[i,j]/np.sqrt(post_cov[i,i]*post_cov[j,j])

#Compute the posterior mean
Xt_Sige_Y = np.dot(X.T,np.dot(obs_cov_inv,df['logOzone']))
Sigb_priormean = np.dot(prior_cov_inv,prior_mean)
post_mean = np.dot(post_cov,Xt_Sige_Y + Sigb_priormean)
print(f'Posterior mean (Const,Solar.R,Wind,Temp)={post_mean}')

#### How do the posterior mean values compare these values to the ones we got above with the OLS approach? With this Bayesian approach, we can also examine the correlations between these parameters. **Recall that there are no correlations in the prior covariance or observation covariance.**

In [ ]:
fig,ax = plt.subplots(1)
g = ax.imshow(post_corr,vmin=-1,vmax=1,cmap=plt.cm.RdBu_r)
plt.colorbar(g,ax=ax)
ax.set_xticks(range(4))
ax.set_xticklabels(['Const','Solar.R','Wind','Temp'])
ax.set_yticks(range(4))
ax.set_yticklabels(['Const','Solar.R','Wind','Temp'])
ax.set_title('Posterior Correlations in Linear Model Coefs')

### Questions
1. How would you interpret correlations in the posterior distribution for the parameters?
2. Even though we assumed uncorrelated prior and observation error covariances, we still ended up with correlations in the posterior. Where were those introduced? 

---
### **EXTRA CREDIT**: Observing System Error
#### Suppose we are evaluating a new observing system for a quantity $y$ that we know well. We believe the observing system may be biased with error distribution $\epsilon \sim N(\mu,\sigma^2)$. Thus we take a bunch of measurements $y_1,y_2,...,y_n$ and examine the errors $\epsilon_i = y-y_i$. If the number of samples is small and the variability among them seems large, a good Bayesian will incorporate a prior guess on $\mu$ and $\sigma^2$ and update it with the new information with Bayes' Rule. We need to create a prior distribution for these parameters as well as a likelihood. 

#### The typical prior model is a Gaussian model for the parameters. Suppose the manufacturer of our observing system tells us they think the mean error is 0.1 and the variance is 3. Being conservative, we decide that the uncertainty in their estimates is 100% as a one standard deviation value of their estimate. The Gaussian prior would then be $$p(\mu,\sigma) \propto \exp\left(-\frac{1}{2(0.1)^2}(\mu-0.1)^2\right)\exp\left(-\frac{1}{2(3)^2}(\sigma-3)^2\right)$$ The natural model for the likelihood of the observed deviations $(\epsilon_i)$ incorporating the parameters $x = (\mu,\sigma^2)$ is  given by the function $$ p((\epsilon_i)|x) = (2\pi\sigma^2)^{-n/2}\exp\left(-\frac{1}{2\sigma^2}\sum_{i=1}^{n}(\epsilon_i-\mu)^2\right).$$ Note that the likelihood is a function of $\mu$ and $\sigma$ and is not in general a probability distribution for a specified set of observations $\epsilon_i$.
#### Our experiment is a "simulation study", meaning that we proceed as follows:
1. Simulate some "observations" with a "truth" parameter set $(\mu_t,\sigma_t)$
2. Construct a prior distribution with parameters $(\mu_o,\sigma_o)$
3. Construct the likelihood function
4. Multiply the likelihood and prior together and renormalize to get the posterior distribution.

*Note*: we are solving this with a sampling approach instead of constructing the continuous version. 

In [ ]:
# Simulate some data
mu = 1   #bias
sig = 10 #error variability
eps = mu + sig*np.random.randn(100)
plt.hist(eps);
plt.title('Sample Distribution of Observed Errors');

#### The prior is the Gaussian with mean (0.1,3) and diagonal covariance matrix with diagonal values (0.1^2,9^2) due to our assumption of 100% uncertainty and independence. 

In [ ]:
# Now let's build a prior distribution for mu and sig
mu_o = 0.1
sig_o = 3

# Sample grid for plotting
mu_g = np.linspace(mu_o-3*sig_o,mu_o+3*sig_o,200)
sig_g = np.linspace(0.0001,20,100)

# Prior Distribution defined on sample grid
prior = np.array([[gaus_pdf(mu_gg,mu_o,mu_o)*gaus_pdf(sig_gg,sig_o,sig_o) for sig_gg in sig_g] for mu_gg in mu_g])
prior = prior/prior.mean()

plt.pcolormesh(sig_g,mu_g,prior)
plt.colorbar()
plt.xlabel(r'$\sigma$')
plt.ylabel(r'$\mu$')
plt.title('Gaussian Prior Distribution $p(\mu,\sigma)$')

#### We can also plot the 2D Gaussian likelihood function $p((\epsilon_i) | \mu; \sigma)$

In [ ]:
def gaus_likelihood(eps,mu=0,sig=1):
    if not isinstance(eps,(list,tuple,np.ndarray)): gl = gaus_pdf(eps,mu=mu,sig=sig)
    gl = 1.
    for ep in eps:
        gl *= gaus_pdf(ep,mu=mu,sig=sig)
    return gl

In [ ]:
# Sample the likelihood function on a grid for plotting
likelihood = np.array([[gaus_likelihood(eps,mu=mu_gi,sig=sig_gi) for sig_gi in sig_g] for mu_gi in mu_g])

In [ ]:
plt.pcolormesh(sig_g,mu_g,likelihood)
plt.colorbar()
plt.title("Likelihood Function",fontsize=18)
plt.xlabel('$\sigma$',fontsize=18)
plt.ylabel('$\mu$',fontsize=18)

#### In this setting, we call $p(\mu,\sigma|(\epsilon_i))$ the posterior distribution since it is what we know after we take the observations $\epsilon_i$ into account. Using Bayes' Rule above,the posterior density is proportional to the product of the likelihood $p((\epsilon_i)|\mu,\sigma)$ and the prior $p(\mu,\sigma).$ To force $p(\mu,\sigma|(\epsilon_i))$ to be a PDF, we can normalize it.

In [ ]:
# Compute the posterior:
posterior = likelihood*prior/np.nanmean(likelihood*prior) #this works because the grid is evenly spaced
plt.pcolormesh(sig_g,mu_g,posterior)
plt.colorbar()
plt.xlabel(r'$\sigma$')
plt.ylabel(r'$\mu$')
plt.title('Posterior Distribution $p(\mu,\sigma|(\epsilon_i))$');

### **Summary** 
- Our goal was to infer the observational error distribution $\epsilon$.
- We assumed the Gaussian model for the prior parameter distribution and likelihood function.
- With the data we collected and Bayes' Theorem, we discovered a distribution for $\mu$ and $\sigma$ that incorporates the manufacturer's information and our measurements.

#### Questions
1. How is the posterior distribution different from the posterior distribution?
2. What does the shape of the likelihood tell us?
3. Does the manufacturer's specified error statistics match our updated information? 

### **Why a distribution for the parameters for the errors? Why not just values?**
#### We recognize that we may have some knowledge on the parameters $\mu$ and $\sigma$ before even looking at the data, like the range of reasonable values that they fall into. The posterior density factors in both the mismatch to the data as well as this range of values and is a more complete description of the uncertainty than just single numbers. With a distribution, we can always find point estimates.
#### The posterior mean parameter set is computed the normal way: $$ \bar{\mu} = \int\int mp(m,s|(\epsilon_i))dsdm $$ $$ \bar{\sigma} = \int\int sp(m,s|(\epsilon_i))dmds $$ We can also look at the mode or maximum of the posterior distribution and find the parameters that yield that value: $$ \mu^*,\sigma^* = \mbox{argmax}\: p(\mu,\sigma|(\epsilon_i))$$

In [ ]:
# Mean of the posterior distribution with marginal distributions
#mu_mean,sig_mean = np.nanmean(mu_g[:,None]*posterior),np.nanmean(sig_g[None]*posterior)
# Mean and Mode of the posterior distribution with marginal distributions
mu_mean,sig_mean = (mu_g*posterior.mean(1)).mean(),(sig_g*posterior.mean(0)).mean()
mu_mode,sig_mode = mu_g[np.argmax((posterior.mean(1)))],sig_g[np.argmax(posterior.mean(0))]

In [ ]:
plt.pcolormesh(sig_g,mu_g,posterior)
g1 = plt.plot([sig_mode],[mu_mode],'r*')
g2 = plt.plot([sig_mean],[mu_mean],'x',color='orange')
plt.colorbar()
plt.legend([g1[0],g2[0]],[f'Mode ($\mu=${mu_mode:4.2f},$\sigma=${sig_mode:4.2f})',f'Mean ($\mu=${mu_mean:4.2f},$\sigma=${sig_mean:4.2f})'])
#plt.legend([g2[0]],[f'Mean ($\mu=${mu_mean:4.2f},$\sigma=${sig_mean:4.2f})'])
plt.xlabel(r'$\sigma$')
plt.ylabel(r'$\mu$')
plt.title('Posterior Distribution $p(\mu,\sigma|(\epsilon_i))$');

#### **Note**: in this problem the two estimates should be identical. The small differences arise from the sampling approach we took to solve the problem. If we represented the distributions from the start with continuous functions, the difference would disappear. 

#### Questions:
1. How do these point estimates compare to what we assumed above?
2. How would changing the assumptions about the prior uncertainties on $\mu$ and $\sigma$ change the results?